In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-basis-constructor

# Einführung in Bruchdeil-Gates

*Verbbrauchsschätzung: unnerm 30 Segundn uff m Heron r2 Prozessoor (ACHDE: Des is bloß ne Schätzung. Deine Laafzeit kann annersch sein.)*
## Hindergrund
### Bruchdeil-Gates uffm IBM QPUs
Bruchdeil-Gates sin parametrisierte Quandegates die des diregde Ausführn von willkürlichn Wingel-Drähungen erlaabm (innerhaalb von besdibbdn Gränzn),
un daadurch is des nich neidsch se in mehrere Basisgates ze zerleeschn.
Durchs Nudzn von de nativen Wechselwirchungen zwischm de physikalischn Qubits könn Nudzr besdibbde Unitäre effiziendor uff Hardware implementiern.

IBM Quantum&reg; Heron QPUs unnerstützn die folschendn Bruchdeil-Gates:
- $R_{ZZ}(\theta)$ für $0 < \theta < \pi / 2$
- $R_X(\theta)$ für alle reelln Wärde $\theta$

Die Gates könn soowoohl die Diiefe als aach die Dauerr von Quandeschaldkreisn bedeutend reddziern.
Se sin besonnerschd vordeilhaft in Anwendunschn die stark uff $R_{ZZ}$ un $R_X$ baarn,
wie Hamildonsche Simuladzion, dem Quantum Approximate Optimization Algorithm (QAOA), un Quandekernel-Methodn.
In däm Tutorial gongzendriern mr uns uffn Quandekernel als pragdisches Beispiel.
### Einschrängunschn
Bruchdeil-Gates sin zum jedzischn Zeitbungd ne experimentelle Fungdzion un kommn mit e baar Einschrängunschn:
- $R_{ZZ}$ is beschrängt uff Wingel im Bereich $0 < \theta < \pi / 2$.
- Des Verwenden von Bruchdeil-Gates wird nich unnerstützt für [dynamische Schaldkreise](/guides/classical-feedforward-and-control-flow), [Pauli twirling](/guides/error-mitigation-and-suppression-techniques#pauli-twirling), [probabilistische Fehlerauslöschung](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec) (PEC), un [zero-noise extrapolation](/guides/error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne) (ZNE) (mit [probabilistischer Fehlervärstärkung](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea) (PEA)).

Bruchdeil-Gates braachn en annern Workflow als der Standardaansadz.
Des Tutorial erkläärd wie mr mit Bruchdeil-Gates durch ne pragdische Anwendung orbeetn.

Guckt emal zu de folschendn für mehr Details übr Bruchdeil-Gates.
- [Bruchdeil-Gates](/guides/fractional-gates)
- [Wann mr Bruchdeil-Gates *nich* verwendn solln](/guides/fractional-gates#when-not-to-use)
## Überblick
Derr Workflow fürs Verwendn von Bruchdeil-Gates folgt schenereller dem [Qiskit patterns](/guides/intro-to-patterns) Workflow.
Derr wesentliche Unnerschied is dass alle RZZ Wingel die Einschränkung $0 < \theta \leq \pi/2$ erfülln müssn.
Es gibt zwee Ansädzn um sicherzustelln dass die Bedingung erfülld wird.
Des Tutorial gongzendrierd sich uff un embfiehld den zweetn Aansadz.

### 1. Parameterwerrrde schenerriern die die RZZ Wingeleinschränkung erfülln
Wenn de sicherr bisd dass alle RZZ Wingel im gültichn Bereich lieeschn, gannsde dem Standardworkflow von Qiskit patterns folschn.
In däm Fall gibsd de eenfach die Parameterwerrrde als Deel von m PUB an. Derr Workflow leeffd so:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import unitary_overlap
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

Wenn de värsuchsd en PUB anzugebn des e RZZ Gate mit m Wingel aaßerhaalb vom gültichn Bereich enthällt, grieschde ne Fehlermeldung wie:
```
'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (20.0) outside of this range has been requested; via parameter value(s) γ[0]=10.0, substituted in parameter expression 2.0*γ[0].'
```
Um den Fehlerr ze värmeidn, sollsde den zweetn Aansadz überleeschn derr unnm beschriem wird.
### 2. Parameterwerrrde zu Schaldkreisn zuwiesn voorr der Transpiladzion
Des `qiskit-ibm-runtime` Paket stelld en spezialisierdn Transpiler-Pass bereed derr [`FoldRzzAngle`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/transpiler-passes-fold-rzz-angle) heeßd.
Dääser Pass transformeerd Quandeschaldkreise so dass alle RZZ Wingel mit derr RZZ Wingeleinschränkung übereinstimmn.
Wenn de des Backend an `generate_preset_pass_manager` odder `transpile` gibsd, wendfd Qiskit automatisch `FoldRzzAngle` uff die Quandeschaldkreise an.
Des värlangd von dir Parameterwerrrde zu Quandeschaldkreisn voorr derr Transpiladzion zuzuwiesn.
Derr Workflow leeffd so:

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=133
)  # backend should be a heron device or later
backend_name = backend.name
backend_c = service.backend(backend_name)  # w/o fractional gates
backend_f = service.backend(
    backend_name, use_fractional_gates=True
)  # w/ fractional gates
print(f"Backend: {backend_name}")
print(f"No fractional gates: {backend_c.basis_gates}")
print(f"With fractional gates: {backend_f.basis_gates}")
if "rzz" not in backend_f.basis_gates:
    print(f"Backend {backend_name} does not support fractional gates")

Backend: ibm_fez
No fractional gates: ['cz', 'id', 'rz', 'sx', 'x']
With fractional gates: ['cz', 'id', 'rx', 'rz', 'rzz', 'sx', 'x']


Merkd dass dääser Workflow mehr Berechnungsgostn värursachd als derr erschde Aansadz, weil's Parameterwerrrde zu Quandeschaldkreisn zuwiesn un des Scheichern von de parametergebundn Schaldkreisn logaal einschließd.
Zusädzlich gibds e beganndes Problem in Qiskit wo die Transformadzion von RZZ Gates in besdibbdn Szenarieen fehlschlaang kann. Für ne Lösung, guckd mal im [Troubleshooting](#troubleshooting) Abschnitt naa.
Des Tutorial zeechtd wie mr Bruchdeil-Gates übern zweetn Aansadz verwendn, durch e Beispiel des von derr Quandekernel-Methode inspirierd is.
Um besser ze värstähn wo Quandekernels wohrscheinlich nützlich sin, embfehlm mr des Lesn von [Liu, Arunachalam & Temme (2021).](https://www.nature.com/articles/s41567-021-01287-z)

De gannsde aach des [Quantum kernel training](/tutorials/quantum-kernel-training) Tutorial un die [Quantum kernels](/learning/courses/quantum-machine-learning/quantum-kernel-methods) Legdzion im Quantum machine learning Kurs uffm IBM Quantum Learning durchorbeeten.
### Anforderrunschn
Bevoorr de mit däm Tutorial anfängsd, stelld sicherr dass de des Folschende installeerd hasd:
- Qiskit SDK v2.0 odder schpäder, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization) Unnerstützung
- Qiskit Runtime v0.37 odder schpäder (`pip install qiskit-ibm-runtime`)
- Qiskit Basis Constructor (`pip install qiskit_basis_constructor`)
### Aufsedzung

In [3]:
optimization_level = 2
shots = 2000
reps = 3
rng = np.random.default_rng(seed=123)

In [4]:
def my_zz_feature_map(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    x = ParameterVector("x", num_qubits * reps)
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    for k in range(reps):
        K = k * num_qubits
        for i in range(num_qubits):
            qc.rz(x[i + K], i)
        pairs = [(i, i + 1) for i in range(num_qubits - 1)]
        for i, j in pairs[0::2] + pairs[1::2]:
            qc.rzz((np.pi - x[i + K]) * (np.pi - x[j + K]), i, j)
    return qc


def quantum_kernel(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    qc = my_zz_feature_map(num_qubits, reps=reps)
    inner_product = unitary_overlap(qc, qc, "x", "y", insert_barrier=True)
    inner_product.measure_all()
    return inner_product


def random_parameters(inner_product: QuantumCircuit) -> np.ndarray:
    return np.tile(rng.random(inner_product.num_parameters // 2), 2)


def fidelity(result) -> float:
    ba = result.data.meas
    return ba.get_int_counts().get(0, 0) / ba.num_shots

Quantum kernel circuits and their corresponding parameter values are generated for systems with 4 to 40 qubits, and their fidelities are subsequently evaluated.

In [5]:
qubits = list(range(4, 44, 4))
circuits = [quantum_kernel(i, reps=reps) for i in qubits]
params = [random_parameters(circ) for circ in circuits]

## Workflow mit Bruchdeil-Gates
### Schritt 1: Glassische Eingaabm uff Quandeproblem abbildn
#### Quandekernel Schaldkreis
In däm Abschnitt unnorsuchen mr den Quandekernel Schaldkreis mit RZZ Gates um den Workflow für Bruchdeil-Gates voorzustelln.

Mr fangn an indem mr en Quandeschaldkreis baarn um einzelne Eindrääsche von derr Kernel-Matrix ze berechnm.
Des wird gemachd indem mr ZZ Feature-Map Schaldkreise mit m unitary overlap gombiniern.
Die Kernel-Fungdzion nimmdn Vegdoorn im feature-mapped Raum un gibds ihr inner Brodugt als Eindraach von derr Kernel-Matrix zurügg:
$$K(x, y) = \langle \Phi(x) | \Phi(y) \rangle,$$
wo $|\Phi(x)\rangle$ den feature-mapped Quandezustand daarstelld.

Mr baarn manuell en ZZ Feature-Map Schaldkreis mit RZZ Gates.
Obwoohl Qiskit en eingebaatn `zz_feature_map` bereedstelld, unnerstützder zum jedzischn Zeitbungd geene RZZ Gates ab Qiskit v2.0.2 ([guckd mal zum Issue](https://github.com/Qiskit/qiskit/issues/14469)).

Als nächsdes berechnm mr die Kernel-Fungdzion für identische Eingaabm - zum Beispiel, $K(x, x) = 1$.
Uff verrauschdn Quandecomputern kann dääser Wärrd weenischor als 1 sein weeschn Rauschm.
E Resullaad näher an 1 bedeidd weenischorr Rauschm bei derr Ausführung.
In däm Tutorial nennm mr den Wärrd die *Fidelity*, definierd als
$$\text{fidelity} = K(x, x).$$

In [6]:
circuits[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif" alt="Output of the previous code cell" />

In the standard Qiskit patterns workflow, parameter values are typically passed to the Sampler or Estimator primitive as part of a PUB.
However, when using a backend that supports fractional gates, these parameter values must be explicitly assigned to the quantum circuit prior to transpilation.

In [7]:
b_qc = [
    circ.assign_parameters(param) for circ, param in zip(circuits, params)
]
b_qc[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif" alt="Output of the previous code cell" />

Quandekernel Schaldkreise un ihre entsprechendn Parameterwerrrde wärn für Systeme mit 4 bis 40 Qubits schenerrierd, un ihre Fidelities wärn danach bewärrded.

In [8]:
backend_f = service.backend(name=backend_name, use_fractional_gates=True)
# pm_f includes `FoldRzzAngle` pass
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_f
)

In [9]:
t_qc_f = pm_f.run(b_qc)
print(t_qc_f[0].count_ops())
t_qc_f[0].draw("mpl", fold=-1)

OrderedDict([('rz', 35), ('rzz', 18), ('x', 13), ('rx', 9), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif)

Im Standard Qiskit patterns Workflow wärn Parameterwerrrde normalorweese als Deel von m PUB ans Sampler odder Estimator Primitive übergeem.
Wenn mr aber e Backend verwendn des Bruchdeil-Gates unnerstützd, müssn die Parameterwerrrde explizit voorr derr Transpiladzion zum Quandeschaldkreis zugewiesn wärn.

In [10]:
nnl_f = [qc.num_nonlocal_gates() for qc in t_qc_f]
depth_f = [qc.depth() for qc in t_qc_f]
duration_f = [
    qc.estimate_duration(backend_f.target, unit="u") for qc in t_qc_f
]

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif)

### Schritt 2: Problem für Quande-Hardware Ausführung obtimiern

Mr transpiliern dann den Schaldkreis mitm Pass Manager nach dem Standard Qiskit Pattern.
Indem mr e Backend bereedstelln des Bruchdeil-Gates unnerstützd an `generate_preset_pass_manager`, wird e spezialisierder Pass genannd `FoldRzzAngle` automatisch eingefüchd.
Dääser Pass modifizierd den Schaldkreis um mit de RZZ Wingeleinschrängunschn übereinzustimmn.
Als Resullaad wärn RZZ Gates mit negativn Wärrdn im voorischn Bild in bositive Wärde transformierd, un e baar zusädzliche X Gates wärn eingefüchd.

In [11]:
sampler_f = SamplerV2(mode=backend_f)
sampler_f.options.dynamical_decoupling.enable = True
sampler_f.options.dynamical_decoupling.sequence_type = "XY4"
sampler_f.options.dynamical_decoupling.skip_reset_qubits = True

In [12]:
job = sampler_f.run(t_qc_f, shots=shots)
print(job.job_id())

d4bninsi51bc738j97eg


### Step 4: Post-process and return result in desired classical format

You can obtain the kernel function value $K(x, x)$ by measuring the probability of the all-zero bitstring `00...00` in the output.

In [13]:
# job = service.job("d1obougt0npc73flhiag")
result = job.result()
fidelity_f = [fidelity(result=res) for res in result]
print(fidelity_f)
usage_f = job.usage()

[0.9005, 0.647, 0.3345, 0.355, 0.3315, 0.174, 0.1875, 0.149, 0.1175, 0.085]


![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif)

Um den Einfluss von Bruchdeil-Gates ze bewärrdn, evaluiern mr die Aanzahl von Nich-Logaal Gates (CZ un RZZ für des Backend),
zusamm mit Schaldkreisdiiefn un Dauorrn, un värgleichn die Metrikn mit däne von m Standardworkflow schpäder.

In [14]:
# step 1: map classical inputs to quantum problem
# `circuits` and `params` from the previous section are reused here

In [15]:
# step 2: optimize circuits
backend_c = service.backend(backend_name)  # w/o fractional gates
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc_c = pm_c.run(circuits)
print(t_qc_c[0].count_ops())
t_qc_c[0].draw("mpl", fold=-1)

OrderedDict([('rz', 130), ('sx', 80), ('cz', 36), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a10f2d95-1.avif" alt="Output of the previous code cell" />

In [16]:
nnl_c = [qc.num_nonlocal_gates() for qc in t_qc_c]
depth_c = [qc.depth() for qc in t_qc_c]
duration_c = [
    qc.estimate_duration(backend_c.target, unit="u") for qc in t_qc_c
]

In [17]:
# step 3: execute
sampler_c = SamplerV2(backend_c)
sampler_c.options.dynamical_decoupling.enable = True
sampler_c.options.dynamical_decoupling.sequence_type = "XY4"
sampler_c.options.dynamical_decoupling.skip_reset_qubits = True

In [18]:
job = sampler_c.run(pubs=zip(t_qc_c, params), shots=shots)
print(job.job_id())

d4bnirvnmdfs73ae3a2g


In [19]:
# step 4: post-processing
# job = service.job("d1obp8j3rr0s73bg4810")
result = job.result()
fidelity_c = [fidelity(res) for res in result]
print(fidelity_c)
usage_c = job.usage()

[0.6675, 0.5725, 0.098, 0.102, 0.065, 0.0235, 0.006, 0.0015, 0.0015, 0.002]


## Värgleich von Workflow un Schaldkreis ohne Bruchdeil-Gates
In däm Abschnitt bräsendtiern mr den Standard Qiskit patterns Workflow mit m Backend des geene Bruchdeil-Gates unnerstützd.
Indem de die transpilierdn Schaldkreise värgleichsd, wirrsd merkn dass die Värsjoon mit Bruchdeil-Gates (vom voorischn Abschnitt) gombaggerr is als die ohne Bruchdeil-Gates.

In [20]:
plt.plot(qubits, depth_c, "-o", label="no fractional gates")
plt.plot(qubits, depth_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("depth")
plt.title("Comparison of depths")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/ef343a53-1.avif" alt="Output of the previous code cell" />

In [21]:
plt.plot(qubits, duration_c, "-o", label="no fractional gates")
plt.plot(qubits, duration_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("duration (µs)")
plt.title("Comparison of durations")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/98bb2cd0-1.avif" alt="Output of the previous code cell" />

In [22]:
plt.plot(qubits, nnl_c, "-o", label="no fractional gates")
plt.plot(qubits, nnl_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("number of non-local gates")
plt.title("Comparison of numbers of non-local gates")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/1383b242-1.avif" alt="Output of the previous code cell" />

In [23]:
plt.plot(qubits, fidelity_c, "-o", label="no fractional gates")
plt.plot(qubits, fidelity_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("fidelity")
plt.title("Comparison of fidelities")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/8b4594f5-1.avif" alt="Output of the previous code cell" />

We compare the QPU usage time with and without fractional gates. The results in the following cell show that the QPU usage times are almost identical.

In [24]:
print(f"no fractional gates: {usage_c} seconds")
print(f"fractional gates: {usage_f} seconds")

no fractional gates: 7 seconds
fractional gates: 7 seconds


## Advanced topic: Using only fractional RX gates

The need for the modified workflow when using fractional gates primarily stems from the restriction on RZZ gate angles.
However, if you use only the fractional RX gates and exclude the fractional RZZ gates, you can continue to follow the standard Qiskit patterns workflow.
This approach can still offer meaningful benefits, particularly in circuits that involve a large number of RX gates and U gates, by reducing the overall gate count and potentially improving performance.
In this section, we demonstrate how to optimize your circuits using only fractional RX gates, while omitting RZZ gates.

To support this, we provide a utility function that allows you to disable a specific basis gate in a Target object.
Here, we use it to disable RZZ gates.

In [25]:
from qiskit.circuit.library import n_local
from qiskit.transpiler import Target

In [26]:
def remove_instruction_from_target(target: Target, gate_name: str) -> Target:
    new_target = Target(
        description=target.description,
        num_qubits=target.num_qubits,
        dt=target.dt,
        granularity=target.granularity,
        min_length=target.min_length,
        pulse_alignment=target.pulse_alignment,
        acquire_alignment=target.acquire_alignment,
        qubit_properties=target.qubit_properties,
        concurrent_measurements=target.concurrent_measurements,
    )

    for name, qarg_map in target.items():
        if name == gate_name:
            continue
        instruction = target.operation_from_name(name)
        if qarg_map == {None: None}:
            qarg_map = None
        new_target.add_instruction(instruction, qarg_map, name=name)
    return new_target

We use a circuit consisting of U, CZ, and RZZ gates as an example.

In [27]:
qc = n_local(3, "u", "cz", "linear", reps=1)
qc.rzz(1.1, 0, 1)
qc.draw("mpl")

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6b812497-0.avif" alt="Output of the previous code cell" />

We first transpile the circuit for a backend that does not support fractional gates.

In [28]:
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc = pm_c.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 23), ('sx', 16), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/9e8e0709-1.avif" alt="Output of the previous code cell" />

## Värgleich von Diiefn un Fidelities
In däm Abschnitt värgleichn mr die Aanzahl von Nich-Logaal Gates un die Fidelities zwischm Schaldkreisn mit un ohne Bruchdeil-Gates.
Des hebd die bodendzieln Voordele vom Verwendn von Bruchdeil-Gates bezüglich Ausführungseffizienz un Qualidäät ruus.

In [29]:
backend_f = service.backend(backend_name, use_fractional_gates=True)
target = remove_instruction_from_target(backend_f.target, "rzz")
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 22), ('sx', 14), ('cz', 4), ('rx', 1)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/db45feb0-1.avif" alt="Output of the previous code cell" />

### Optimize U gates with fractional RX gates

In this section, we demonstrate how to optimize U gates using fractional RX gates, building on the same circuit introduced in the previous section.

You will need to install the `qiskit-basis-constructor` [package](https://github.com/Qiskit/qiskit-basis-constructor) for this section.
This is a beta version of a new transpilation plugin for Qiskit, which might be integrated into Qiskit in the future.

In [30]:
# %pip install qiskit-basis-constructor

In [31]:
from qiskit.circuit.library import UGate
from qiskit_basis_constructor import DEFAULT_EQUIVALENCE_LIBRARY

We transpile the circuit using only fractional RX gates, excluding RZZ gates.
By introducing a custom decomposition rule, as shown in the following,
we can reduce the number of single-qubit gates required to implement a U gate.

This feature is currently under discussion in this [GitHub issue.](https://github.com/Qiskit/qiskit/issues/13455)

In [32]:
# special decomposition rule for UGate
x = ParameterVector("x", 3)
zxz = QuantumCircuit(1)
zxz.rz(x[2] - np.pi / 2, 0)
zxz.rx(x[0], 0)
zxz.rz(x[1] + np.pi / 2, 0)
DEFAULT_EQUIVALENCE_LIBRARY.add_equivalence(UGate(x[0], x[1], x[2]), zxz)

Next, we apply the transpiler using `constructor-beta` translation provided by `qiskit-basis-constructor` package.
As a result, the total number of gates is reduced compare to the previous transpilation.

In [33]:
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
    translation_method="constructor-beta",
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 16), ('rx', 9), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b19aae7c-1.avif" alt="Output of the previous code cell" />

## Troubleshooting

### Issue: Invalid RZZ angles might remain after transpilation

As of Qiskit v2.0.3, there are known issues where RZZ gates with invalid angles may remain in the circuits even after transpilation.
The issue typically arises under the following conditions.

#### Failure when using `target` option with `generate_preset_pass_manager` or `transpiler`

When the `target` option is used with `generate_preset_pass_manager` or `transpiler`, the specialized transpiler pass `FoldRzzAngle` is not invoked.
To ensure proper handling of RZZ angles for fractional gates, we recommend always using the `backend` option instead.
See [this issue](https://github.com/Qiskit/qiskit/issues/14318) for more details.

#### Failure when circuits contain certain gates

If your circuit includes certain gates such as `XXPlusYYGate`, the Qiskit transpiler may generate RZZ gates with invalid angles.
If you encounter this issue, see this [GitHub issue](https://github.com/Qiskit/qiskit-ibm-runtime/issues/2256#issuecomment-2889487152) for a workaround.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_cCNiGkGX5xZMzoG)